In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
data = [
    [True, True, False, True, True],
    [False, True, False, False, True],
    [True, True, False, True, False],
    [True, True, True, False, True],
    [True, False, False, False, True],
]

import numpy as np

np.append(np.array(data).flatten(), True)

array([ True,  True, False,  True,  True, False,  True, False, False,
        True,  True,  True, False,  True, False,  True,  True,  True,
       False,  True,  True, False, False, False,  True,  True])

In [ ]:
from agent import DQNAgent
import asyncio
import websockets
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from agent import DQNAgent
import numpy as np

In [4]:
masked = torch.tensor(
    [
        [
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            -6.6584e-02,
        ]
    ]
)

print(masked)

masked.argmax()

tensor([[ 1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
         -6.6584e-02]])


tensor(0)

In [5]:
a = torch.randn(1, 26)
print(a)
a.argmax().item()

tensor([[ 1.2704,  1.3204, -1.7562, -0.1587, -0.8209,  0.1939, -1.3170, -0.8240,
         -0.3262, -0.4590,  0.1960,  0.0608,  0.0066, -0.5254, -0.7496, -0.2367,
         -0.8550,  0.1997,  0.4256,  1.0672, -1.6388, -0.8028, -0.2542, -1.0595,
          0.4138, -0.5352]])


1

In [6]:
batch_size = 32
channels = 1
board_height = 5
board_width = 5
tensor_cnn = torch.tensor([batch_size, channels, board_height, board_width])

In [ ]:
a = torch.zeros([1, 5, 5])
print(a)

b = torch.ones([2, *a.shape])
b[0] = a
b

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]])

In [ ]:
def preprocess_state(board: list[str]):
    """
    Convert the board (list of strings, e.g. ["XX.O.", ...]) into a float tensor.
    For example:
    'X' -> 1.0  (Black)
    'O' -> -1.0 (White)
    '.' -> 0.0  (Empty)
    '#' -> 0.0  (Dead node, treat as non-playable)
    etc.
    ------------------------------
    """
    w = 5
    h = 5
    new_tensor = torch.zeros([1, 1, w, h], dtype=torch.float)
    for x in range(w):
        for y in range(h):
            ch = board[x][y]
            if ch == "X":
                new_tensor[0][0][x][y] = 1.0
            elif ch == "Y":
                new_tensor[0][0][x][y] = -1.0
            else:
                new_tensor[0][0][x][y] = 0.0

    return new_tensor  # shape [1, 1, 5, 5]

In [9]:
%%timeit
preprocess_state(
    [
        "#X...",
        "#Y...",
        "#....",
        "#....",
        "#....",
    ]
)

145 μs ± 1.34 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
def preprocess_state2(board_state_strs):
    """
    Convert the board (list of strings, e.g. ["XX.O.", ...]) into a float tensor of shape [1,1,w,h].
    'X' -> 1.0  (Black)
    'O' -> -1.0 (White)
    '.' -> 0.0  (Empty)
    '#' -> 0.0  (Dead node, treat as non-playable, will be masked out later anyways)
    """

    w, h = 5, 5
    board_2d = []
    for x in range(w):
        row = []
        for y in range(h):
            ch = board_state_strs[x][y]
            if ch == "X":
                row.append(1.0)
            elif ch == "Y":
                row.append(-1.0)
            else:
                row.append(0.0)
        board_2d.append(row)

    # Convert to Tensor shape [5, 5]
    board_tensor = torch.tensor(board_2d, dtype=torch.float)
    # Add batch and channel dims => shape [1, 1, 5, 5]
    board_tensor = board_tensor.unsqueeze(0).unsqueeze(0)

    return board_tensor

In [11]:
preprocess_state2(
    [
        "#X...",
        "#Y...",
        "#....",
        "#....",
        "#....",
    ]
)

tensor([[[[ 0.,  1.,  0.,  0.,  0.],
          [ 0., -1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]]])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

a = torch.tensor([1, 2], device=device)

In [24]:
def preprocess_state(board_state: list[str], history: list[list[str]]):
    """
    Convert the board (list of strings, e.g. ["XX.O#", ...]) into a float tensor of shape [1,7,w,h].
    '#' -> 1.0  Channel 0
    'X' -> 1.0  Black, Channel 1, 3, 5
    'O' -> 1.0  White, Channel 2, 4, 6
    '.' -> 0.0  (Empty)
    """
    w, h = 5, 5

    channels = []

    # disabled channel
    disabled_channel = torch.zeros(w, h)
    current_black = torch.zeros(w, h)
    current_white = torch.zeros(w, h)

    for x in range(w):
        for y in range(h):
            ch = board_state[x][y]
            if ch == "X":
                current_black[x][y] = 1.0
            elif ch == "O":
                current_white[x][y] = 1.0
            elif ch == "#":
                disabled_channel[x][y] = 1.0

    print(disabled_channel)
    print(current_black)
    print(current_white)
    channels.extend([disabled_channel, current_black, current_white])
    # board_tensor = torch.stack([disabled_channel, current_black, current_white])

    for past_idx in range(2):
        past_step = history[past_idx]

        history_black = torch.zeros(w, h)
        history_white = torch.zeros(w, h)
        for x in range(w):
            for y in range(h):
                ch = past_step[x][y]
                if ch == "X":
                    history_black[x][y] = 1.0
                elif ch == "O":
                    history_white[x][y] = 1.0

        # board_tensor = torch.stack([board_tensor, history_black, history_white])
        channels.extend([history_black, history_white])

    board_tensor = torch.stack(channels).unsqueeze(0)

    return board_tensor

In [26]:
preprocess_state(
    [".....", "..X..", ".....", "..OO.", "..X.."],
    [
        [".....", "..X..", ".....", "...O.", "..X.."],
        [".....", "..X..", ".....", "...O.", "....."],
        [".....", "..X..", ".....", ".....", "....."],
        [".....", ".....", ".....", ".....", "....."],
    ],
).shape

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0.]])


torch.Size([1, 7, 5, 5])